In [1]:
import pandas as pd 

from tqdm.notebook import tqdm

import sys 
sys.path.append('..')

In [2]:
metadata = pd.read_csv('../data/metadata.csv')
print(metadata.shape)
metadata.head()

(5902, 9)


,image_id,path,height,width,viewpoint,date,timestamp,whale_id,encounter_id
0,train0000,images/train0000.jpg,463,150,top,2017-08-07,2017-08-07 20:38:36,whale000,whale000-000
1,train0001,images/train0001.jpg,192,81,top,2019-08-05,2019-08-05 16:49:13,whale001,whale001-000
2,train0002,images/train0002.jpg,625,183,top,2017-08-07,2017-08-07 22:12:19,whale002,whale002-000
3,train0003,images/train0003.jpg,673,237,top,2017-08-07,2017-08-07 20:40:59,whale003,whale003-000
4,train0004,images/train0004.jpg,461,166,top,2018-08-10,2018-08-10 21:45:30,whale004,whale004-000


In [24]:
import torch
from torchvision import transforms as T
from torchvision.io import read_image

import torch.nn as nn

image_size = 256

class BellugaDataset(torch.utils.data.Dataset):
    def __init__(self, metadata):
        
        self.metadata = metadata
        self.metadata['idx'] = self.metadata.index
        
        self.rootDir = '../data/'
        
        self.transforms = torch.nn.Sequential(
            T.Resize([image_size, image_size]),
            T.ConvertImageDtype(torch.float)
        )
        

    def __len__(self):
        return metadata.shape[0]
    
    def __getitem__(self, idx):

        path = self.metadata.path[idx]
        
        image = self.transforms(read_image(self.rootDir + path))
        
        return image

In [25]:
train_dataset = BellugaDataset(metadata)

image_dataloader = torch.utils.data.DataLoader(
                        train_dataset, 
                        batch_size=1,
                        shuffle=True, 
                        num_workers=0
                    )

In [26]:
####### COMPUTE MEAN / STD

# placeholders
psum    = torch.tensor([0.0, 0.0, 0.0])
psum_sq = torch.tensor([0.0, 0.0, 0.0])

# loop through images
for inputs in tqdm(image_dataloader):
    psum    += inputs.sum(axis        = [0, 2, 3])
    psum_sq += (inputs ** 2).sum(axis = [0, 2, 3])

  0%|          | 0/5902 [00:00<?, ?it/s]

In [27]:
####### FINAL CALCULATIONS

# pixel count
count = len(metadata) * image_size * image_size

# mean and std
total_mean = psum / count
total_var  = (psum_sq / count) - (total_mean ** 2)
total_std  = torch.sqrt(total_var)

# output
print('mean: '  + str(total_mean))
print('std:  '  + str(total_std))

mean: tensor([0.4234, 0.4272, 0.4641])
std:  tensor([0.2037, 0.2027, 0.2142])


In [30]:
path = metadata.path[0]
        
image = train_dataset.transforms(read_image('../data/' + path))
image

tensor([[[0.2941, 0.3098, 0.3333,  ..., 0.1961, 0.2000, 0.2039],
         [0.3569, 0.3686, 0.3843,  ..., 0.2157, 0.2118, 0.2078],
         [0.3961, 0.3804, 0.3569,  ..., 0.2078, 0.2118, 0.2118],
         ...,
         [0.9961, 0.9961, 0.9961,  ..., 0.2157, 0.2157, 0.2118],
         [1.0000, 1.0000, 1.0000,  ..., 0.2039, 0.2078, 0.2078],
         [1.0000, 1.0000, 1.0000,  ..., 0.1098, 0.1255, 0.1333]],

        [[0.3137, 0.3294, 0.3490,  ..., 0.2039, 0.2078, 0.2118],
         [0.3686, 0.3804, 0.3961,  ..., 0.2235, 0.2275, 0.2275],
         [0.3922, 0.3765, 0.3529,  ..., 0.2078, 0.2157, 0.2196],
         ...,
         [0.9961, 0.9961, 0.9961,  ..., 0.2000, 0.2000, 0.1961],
         [1.0000, 1.0000, 1.0000,  ..., 0.1882, 0.1922, 0.1922],
         [1.0000, 1.0000, 1.0000,  ..., 0.1020, 0.1098, 0.1176]],

        [[0.3373, 0.3529, 0.3765,  ..., 0.1922, 0.1961, 0.2000],
         [0.3882, 0.3961, 0.4157,  ..., 0.2078, 0.2118, 0.2118],
         [0.4118, 0.3961, 0.3725,  ..., 0.2000, 0.2039, 0.